In [ ]:
import sys, os
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import pickle

sys.path.append("..")
from src.data.buckets import MakeBuckets

In [ ]:
with open("../1.make_tokenizer/smiles_tokenizer.pkl", mode='rb') as f:
    tokenizer = pickle.load(f)
PAD = tokenizer.pad_token

In [ ]:
# Modify according to your dataset.
smiles_zinc_df = pd.read_csv("/cddd/220215_zinc_processed.txt", sep='\t', index_col=0)
random_smiles = smiles_zinc_df['randomized_smiles']
canonical_smiles = smiles_zinc_df['canonical_smiles']

# Make dataset

In [ ]:
random_tokens = [tokenizer.tokenize(smiles) for smiles in tqdm(smiles_zinc_df['randomized_smiles'])]
canonical_tokens = [tokenizer.tokenize(smiles) for smiles in tqdm(smiles_zinc_df['smiles'])]
bucket = MakeBuckets([random_tokens, canonical_tokens], [],[], 10, 120, 10, pad_token=PAD, sub_pad_tokens=[], min_n_token=50000, residue='include')


In [ ]:
splits = bucket.split([0.9997, 0.0003], split_array=True)
for split in splits:
    split.shuffle_batch(n_tokens=12500, n_epoch=1)
splits[0].save("train")
splits[1].save("val")